In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
# 列显示全
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

# 读取数据集后合并一起做特征工程

In [14]:
train_path = '../dataset/train.csv'
testA_path = '../dataset/testA.csv'

train = pd.read_csv(train_path)
testA = pd.read_csv(testA_path)

y_true = train['isDefault']
train.drop(['isDefault'], axis=1, inplace=True)
all_data = train.append(testA)

- 样本空值数量特征

In [15]:
# 空值数量特征
all_data['nan_count'] = all_data.isnull().sum(axis=1)
# 空值数量百分比特征
all_data['nan_ration'] = all_data['nan_count'] / 46
all_data['nan_ration'] = all_data['nan_ration'].apply(lambda x : round(x,2))

- LabelEncoder()

In [16]:
# 找出类型为object的列，做LabelEncoder()
object_col = []
for index, value in all_data.dtypes.items():
    if value == 'object':
        object_col.append(index)

# object_col = ['grade', 'subGrade', 'employmentLength', 'issueDate', 'earliesCreditLine']
all_data[object_col] = all_data[object_col].apply(LabelEncoder().fit_transform)

# 数值分布跨度较大的连续型特征做MinMaxScaler()
continue_col = ['loanAmnt','interestRate','installment','employmentTitle','annualIncome','revolBal','title']
all_data[continue_col] = MinMaxScaler().fit_transform(all_data[continue_col])

- 处理缺失值

In [17]:
# 用-1填充空值让模型捕捉到空值信息
all_data = all_data.fillna(-1)

- 特征工程处理后将训练集和测试集分开

In [18]:
train_process = all_data[all_data['id']<800000]
testA_process = all_data[all_data['id']>=800000]

train_process['isDefault'] = y_true.values

<ipython-input-18-0bd2d6126d37>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_process['isDefault'] = y_true.values


In [19]:
train_process.to_csv('../dataset/train_process.csv', index=False)
testA_process.to_csv('../dataset/testA_process.csv', index=False)